# Getting data from nike.com
---

In [1]:
# Modules import 
import pandas as pd
import requests as req
import json
import re

In [2]:
# Product page 
uri = 'https://www.nike.com/w/mens-shoes-nik1zy7ok'
url = 'https://www.nike.com/gb/launch?s=upcoming'
base_url = 'https://api.nike.com'
session = req.Session()

In [3]:
# A function for retrieving lazily loaded products
def get_lazy_products(stub, products):
    response = session.get(base_url + stub).json()
    next_products = response['pages']['next']
    products += response['objects']
    if next_products:
        get_lazy_products(next_products, products)
    return products

In [4]:
# find INITIAL_REDUX_STATE
html_data = session.get(uri).text
redux = json.loads(re.search(r'window.INITIAL_REDUX_STATE=(\{.*?\});', html_data).group(1))

In [5]:
html_data

'\n    <!doctype html>\n    \n    \n          \n      <!-- segmenter ran at 2021-10-18T13:55:36.783Z -->\n      \n      \n\n      \n        \n      \n    \n          \n\n<script>\nif (!(() =>\n            document.cookie.includes(\'audience_segmentation_performed\'))([])) {\n  (fragmentPath => {\n  const error = new Error(\n    // eslint-disable-next-line prefer-template\n    \'ESI fragment \' + fragmentPath + \' requested, but not found in client\',\n  );\n  // eslint-disable-next-line no-undef\n  if (window.newrelic) {\n    console.warn(error);\n    // eslint-disable-next-line no-undef\n    newrelic.noticeError(error, {\n      type: \'ESI_LOAD_ERROR\',\n      fragment: fragmentPath,\n    });\n  } else {\n    throw error; // let another monitoring system catch the error.\n  }\n})(\'https://www.nike.com/fragments/audience\')\n} else {\n  (fragmentPath => {\n  // eslint-disable-next-line no-undef\n  if (window.newrelic) {\n    // eslint-disable-next-line no-undef\n    newrelic.addPageAc

In [6]:
type(html_data)

str

In [7]:
# find the initial products and the api entry point for the recursive loading of additional products
wall = redux['Wall']
initial_products = re.sub('anchor=[0-9]+', 'anchor=0', wall['pageData']['next'])

In [8]:
# find all the products
products = get_lazy_products(initial_products, [])

In [9]:
# Optional: filter by id to get a list with unique products
cloudProductIds = set()
unique_products = []
for product in products:
    try:
        if not product['id'] in cloudProductIds:
            cloudProductIds.add(product['id'])
            unique_products.append(product)
    except KeyError:
        print(product)